In [1]:
"""
Notebook for exploring, testing, etc
To be deleted before deadline

I'm committing this file just in case anything here might be useful
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [2]:
data = pd.read_csv('creditcard.csv')
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [3]:
# Warning/Error message unless type conversion is applied
fraud = data[data['Class'].astype('int') == 1]
normal = data[data['Class'].astype('int') == 0]

In [4]:
# We have 492 frauds out of 284,807 transactions,
# which is highly unbalanced, so
# We'll be using 492 frauds + fraction of normal data
frac = 1 # Adjust fraction here
n = len(fraud) * frac
normalfrac = normal.sample(n=n)

In [14]:
newdata = pd.concat([normalfrac, fraud])
newdata

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
86519,61269.0,0.885878,-1.058506,0.548628,0.555546,-1.206341,0.010307,-0.537782,0.068328,-0.951507,...,-0.095646,-0.299881,-0.214982,-0.039751,0.307054,-0.316614,0.022171,0.059077,216.00,0
126487,78017.0,1.272287,0.319855,-0.063514,0.441530,0.269658,-0.110140,0.025806,-0.040381,0.009267,...,-0.309233,-0.828628,0.048770,-0.793196,0.272502,0.174470,-0.004401,0.020021,1.98,0
189393,128380.0,2.069460,-0.115725,-1.501167,0.475995,-0.014383,-1.260147,0.247858,-0.345127,0.856526,...,0.361446,1.290009,-0.135861,0.039153,0.504099,0.120130,-0.027763,-0.070525,1.00,0
203969,135054.0,2.162855,-1.809679,-0.014828,-1.590185,-2.008016,-0.041967,-1.942686,0.192547,-0.710117,...,-0.072524,0.191109,0.281636,-0.391546,-0.589025,-0.234035,0.050444,-0.039980,40.00,0
178626,123711.0,1.752427,-0.920036,0.505118,1.492066,-1.221809,0.896693,-1.418373,0.528369,1.990028,...,0.211186,0.805725,0.053006,-0.524363,-0.243345,-0.512782,0.094077,-0.025750,61.80,0
65513,51621.0,1.271208,-0.713256,0.388830,-0.801731,-1.001461,-0.515936,-0.609721,-0.017639,-0.945532,...,-0.035905,-0.409133,0.032943,-0.036359,0.210805,-0.481343,-0.007399,0.018526,68.23,0
3893,3464.0,-0.493907,0.450135,1.732650,-0.046726,0.007283,0.584876,0.363987,0.284447,0.108552,...,0.221412,0.506079,-0.021153,-0.806112,-0.103930,-0.478681,0.078910,0.055965,79.60,0
54735,46593.0,0.949922,0.040575,1.437259,2.710100,-0.615720,0.663127,-0.496221,0.245988,-0.007371,...,-0.008997,0.040834,0.006156,0.098328,0.223312,-0.023634,0.050926,0.045878,69.20,0
168384,119202.0,2.071844,-0.980969,-1.355910,-0.992165,-0.563328,-0.800754,-0.411611,-0.144660,-0.561946,...,-0.001417,-0.243297,0.214723,-0.441864,-0.272317,-0.422912,-0.053207,-0.062957,70.00,0
218956,141539.0,0.649911,-2.551270,-1.188172,0.701388,-1.152538,-0.052096,0.403706,-0.155364,0.630939,...,0.192319,-0.921602,-0.224275,-0.298637,-0.871818,0.157486,-0.170800,0.064458,699.00,0


In [6]:
X = newdata.drop(columns='Class').values
y = newdata['Class'].values
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [7]:
bayes_model = GaussianNB()
bayes_model.fit(X_train, y_train)
print(bayes_model.score(X_train, y_train))
print(bayes_model.score(X_valid, y_valid))

0.8821138211382114
0.8495934959349594


In [8]:
bayes_model2 = make_pipeline(
    StandardScaler(),
    GaussianNB()
)
bayes_model2.fit(X_train, y_train)
print(bayes_model2.score(X_train, y_train))
print(bayes_model2.score(X_valid, y_valid))

0.9186991869918699
0.9186991869918699


In [9]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, y_train)
print(knn_model.score(X_train, y_train))
print(knn_model.score(X_valid, y_valid))

0.8021680216802168
0.6463414634146342


In [10]:
knn_model2 = make_pipeline(
    StandardScaler(),
    GaussianNB()
)
knn_model2.fit(X_train, y_train)
print(knn_model2.score(X_train, y_train))
print(knn_model2.score(X_valid, y_valid))

0.9186991869918699
0.9186991869918699


In [11]:
svm_model = SVC(C=1.0, kernel='linear')
svm_model.fit(X_train, y_train)
print(svm_model.score(X_train, y_train))
print(svm_model.score(X_valid, y_valid))

0.9065040650406504
0.8861788617886179


In [12]:
svm_model2 = make_pipeline(
    StandardScaler(),
    SVC(C=1.0, kernel='linear')
)
svm_model2.fit(X_train, y_train)
print(svm_model2.score(X_train, y_train))
print(svm_model2.score(X_valid, y_valid))

0.959349593495935
0.9349593495934959
